In [12]:
from s2h_utils import *
import time

In [40]:
!pip install st-radial

ERROR: Could not find a version that satisfies the requirement st-radial (from versions: none)
ERROR: No matching distribution found for st-radial


J'ai gardé ici les quelques étapes pour resoudre l'exercice grace aux differents reviews que j'ai faits

## Load data

In [36]:
data  = pd.read_excel(r"C:\Users\sawal\OneDrive - UPEC\Diot-Siaci\spro-main\Tests\Exercice_2_Arslene_Abdi\data_at.xlsx")[["id", "DT_DEB_PER_INA","DT_FIN_PER_INA", "LIB_MTF_PER_INA"]]
data.columns = ["id", "date_debut", "date_fin", "motif"]
data['date_debut'] = pd.to_datetime(data['date_debut'],  format = '%Y-%m-%d')
data['date_fin'] = pd.to_datetime(data['date_fin'],  format='%Y-%m-%d')

# External data

In [37]:
external_df = data
external_df['date'] = external_df.apply(lambda x: pd.date_range(x.date_debut, x.date_fin, freq='1d'), axis=1)
# explode  and drop duplicates: from wide to long data
external_df = external_df.explode('date')[["id","date", "motif"]]
external_df = external_df.drop_duplicates()
external_df.to_pickle("./OUTPUT/presence_external.pkl")

## REORGANISATION et traitements des anomalies

In [29]:
start = time.time()
data = reorganisation_all(data)
data.to_pickle("./OUTPUT/presence_anomalies.pkl")

The algorithm stopped after 4 iterations


## generation des tables de presence par jours & aggrégé par mois

In [38]:
dates_presence = data
dates_presence['date'] = dates_presence.apply(lambda x: pd.date_range(x.date_debut, x.date_fin, freq='1d'), axis=1)	
# explode  and drop duplicates: from wide to long data
dates_presence = dates_presence.explode('date')[["id","date"]].drop_duplicates()
dates_presence['presence'] = 1

# GENERATE CALENDAR DATES GIVEN MIN  AND MAX  DATES
baseline = data
baseline = baseline.groupby("id").agg({'date_debut':'min', 'date_fin':'max'}).reset_index()
baseline['date'] = baseline.apply(lambda x: pd.date_range(x.date_debut, x.date_fin, freq='1d'), axis=1)
baseline= baseline.explode('date')[["id","date"]].drop_duplicates()
baseline = baseline[baseline['date'].dt.dayofweek<5] # REMOVING SATURDAY AND SUNDAY

# JOIN two datasets 
dates_presence = pd.merge(baseline, dates_presence, on =['date', 'id'], how='left')
dates_presence['presence'] = dates_presence['presence'].fillna(0)
dates_presence["year_month"] = dates_presence.date.dt.strftime("%Y-%m")
dates_presence_agg = dates_presence.groupby(["year_month", "id"]).agg({'presence' : 'sum'}).reset_index().sort_values("id")
dates_presence_agg["presence"] = dates_presence_agg["presence"].astype(int)

In [39]:
dates_presence = pd.merge(dates_presence, pd.read_pickle( "./OUTPUT/presence_external.pkl"),
         on =['id', 'date'],
        how="left").drop_duplicates(["id","date","presence"])

dates_presence.motif  = dates_presence.motif.fillna("NA")
dates_presence.to_pickle("./OUTPUT/presence_indcator.pkl")

In [9]:
dates_presence.date.dt.dayofweek<5

0         False
1          True
2          True
3          True
4          True
          ...  
364326     True
364327     True
364328     True
364329    False
364330    False
Name: date, Length: 361554, dtype: bool

In [55]:
import plotly.express as px
px.bar(dates_presence.groupby(["year_month", "motif"]).count()["mois"].reset_index(), x="year_month", y="mois", color="motif", title="Long-Form Input")

In [46]:
# 1) REORGANISATION DU DATASET INITIAL PAR MOIS D'ABSCENCE : WIDE TO LONG
data["days_off"] = data.apply(lambda p: get_days_month(p),axis=1)
tmp = data.explode("days_off").reset_index()                          
tmp = pd.concat([tmp.drop(["date_debut", "date_fin"],axis=1) ,
                pd.DataFrame(tmp["days_off"].values.tolist(), columns =['date_debut', 'date_fin'])], axis=1)

# 2) INCREMENTER CHAQUE DATE DE DEBUT D' UN JOURS SI NOUS AVONS PLUSIEURS PERIODES DE CONGEES SUR PLUSIEURS MOIS
tmp["debut_fin_shift"] = tmp.groupby("id")["date_fin"].shift(1)
tmp['date_debut'] =np.where(tmp.debut_fin_shift==tmp.date_debut, tmp.date_debut + pd.DateOffset(days=1), tmp.date_debut)

# 3)CORRECTION DES ANOMALIES
tmp = reorganisation_all(tmp)

# 4)ENRICHIR LA BASE PRECEDENTE AVEC LES DONNNES AGGREGES D'ABSCENCE
tmp["year_month"] =tmp["date_debut"].dt.strftime("%Y-%m")
tmp = pd.merge(dates_presence_agg, tmp, on =['id','year_month'],how='left')
tmp["mois"] = pd.to_datetime(tmp.year_month+'-01')
tmp['NB_arret'] = (((tmp['date_fin'] - tmp['date_debut']).dt.days +1).fillna(0)).astype(int)
tmp

The algorithm stopped after 2 iterations


,year_month,id,presence,date_debut,date_fin,mois,NB_arret
0,2020-06,6.541784e+15,0,NaT,NaT,2020-06-01,0
1,2019-12,6.541784e+15,0,NaT,NaT,2019-12-01,0
2,2020-02,6.541784e+15,0,NaT,NaT,2020-02-01,0
3,2020-09,6.541784e+15,3,2020-09-28,2020-09-30,2020-09-01,3
4,2021-11,6.541784e+15,6,2021-11-25,2021-11-30,2021-11-01,6
...,...,...,...,...,...,...,...
13804,2016-10,1.841011e+19,31,2016-10-02,2016-10-31,2016-10-01,30
13805,2014-12,1.841011e+19,31,2014-12-01,2014-12-01,2014-12-01,1
13806,2014-12,1.841011e+19,31,2014-12-02,2014-12-31,2014-12-01,30
13807,2018-11,1.841011e+19,30,2018-11-01,2018-11-01,2018-11-01,1


In [47]:
end = time.time()
print("Running time in secondes :",end-start)

Running time in secondes : 244.2612612247467


In [48]:
tmp.NB_arret.sum()

46485

In [49]:
tmp = tmp.sort_values(by = ['id', 'mois', 'date_debut', 'date_fin']).reset_index(drop=True)[['id','mois' , 'date_debut', 'date_fin', 'NB_arret']]
tmp.to_pickle("./OUTPUT/presence_agg.pkl")

,year_month,id,presence,date_debut,date_fin,mois,NB_arret
29,2018-06,6.541784e+15,8,2018-06-17,2018-06-24,2018-06-01,8
7,2018-07,6.541784e+15,0,NaT,NaT,2018-07-01,0
44,2018-08,6.541784e+15,0,NaT,NaT,2018-08-01,0
8,2018-09,6.541784e+15,0,NaT,NaT,2018-09-01,0
21,2018-10,6.541784e+15,0,NaT,NaT,2018-10-01,0
...,...,...,...,...,...,...,...
13703,2019-01,1.841011e+19,31,2019-01-02,2019-01-31,2019-01-01,30
13735,2019-02,1.841011e+19,28,2019-02-01,2019-02-01,2019-02-01,1
13736,2019-02,1.841011e+19,28,2019-02-02,2019-02-28,2019-02-01,27
13739,2019-03,1.841011e+19,31,2019-03-01,2019-03-01,2019-03-01,1
